## Questão 1

In [61]:
import numpy as np

def escalona_inteira(A):
    A_ = A.copy()
    m, n = len(A_), len(A_[0])
    n_pivos = 0
    j = 0

    while j < n and n_pivos < m:
        L = [i for i in range(n_pivos, m) if A_[i][j] != 0]

        while len(L) >= 2:
            p = min(L, key=lambda i: abs(A_[i][j]))

            if A_[p][j] < 0:
                for x in range(n):
                    A_[p][x] *= -1

            for k in L:
                if k != p:
                    if A_[k][j] > 0:
                        for i in range(n):
                            A_[k][i] -= A_[p][i]
                    else:
                        for i in range(n):
                            A_[k][i] += A_[p][i]

            L = [i for i in range(n_pivos, m) if A_[i][j] != 0]

        if len(L) == 1:
            p = L[0]
            n_pivos += 1

        j += 1

    return A_

In [62]:
M = [[2, 3, 5, -6],
     [0, -9, 1, 4],
     [7, 0, -2, 3],
     [-8, 3, 0, 4]]
M_ = np.array(escalona_inteira(M))

print(M_)

[[   1   12   22  -27]
 [   0    3  -22   12]
 [   0    0    1 -424]
 [   0    0    0  860]]


## Questão 2

Querido professor, nessa questão eu decidi adaptar o código da questão anterior por 

Nesta questão usar o código do arquivo `escalonamento (3).py` para resolver, usar a mesma lógica utilizada na última questão dessa lista. Nessa questão A*A_ = I (Identidade)

In [63]:
def escalona_matriz(A, inv):
    M = A.copy()
    INV = inv.copy()

    n = len(M)
    
    linha = 0
    while linha < n:
        if (M[linha, linha] == 0):
            i = 1
            while True:
                # Verifica se ainda é possível encontrar a inversa da matriz
                if linha == n or linha+i >= n:
                    print("Impossível encontrar a inversa!")
                    raise "Impossível encontrar a inversa!"
                
                if abs(M[linha+i][linha]) > 2**(-30):
                    # Troca a linha com a linha seguinte
                    aux = M[linha].copy()
                    M[linha] = M[linha+i].copy()
                    M[linha+i] = aux.copy()

                    aux = INV[linha].copy()
                    INV[linha] = INV[linha+i].copy()
                    INV[linha+i] = aux.copy()
                    break
                else:
                    i+=1

        else:  
            mult = M[linha, linha]

            # Vai normalizar a linha colocando o pivo como 1
            for i in range (n):
                M[linha][i] /= mult
                INV[linha][i] /= mult

            # Vai zerar os valores embaixo do pivô
            for i in range(linha+1, n):
                mult = M[i][linha]/M[linha][linha]

                M[i][:] -= mult * M[linha][:]
                INV[i][:] -= mult * INV[linha][:]

            linha += 1

    linha = n-1
    while (linha >= 0):

        for i in range(linha-1, -1, -1):
            mult = M[i, linha]

            M[i][:] -= mult*M[linha][:]
            INV[i][:] -= mult * INV[linha][:]

        linha -= 1

    return INV
        

In [64]:
def calcula_inversa (A):
    T = A.copy().astype(float)

    m = len(T)

    inv = np.identity(m)

    inversa = escalona_matriz(T, inv)

    return inversa

In [65]:
matriz = np.array([[0, 3, 1],
                   [0, 0, 5],
                   [3, 0, 2]])

inversa_matriz = calcula_inversa(matriz)

print(f"Matriz inversa experada: \n{np.linalg.inv(matriz)}")
print(f"\nMatriz inversa calculada: \n{inversa_matriz}\n")

print(matriz@inversa_matriz)

Matriz inversa experada: 
[[ 0.         -0.13333333  0.33333333]
 [ 0.33333333 -0.06666667  0.        ]
 [ 0.          0.2         0.        ]]

Matriz inversa calculada: 
[[ 0.         -0.13333333  0.33333333]
 [ 0.33333333 -0.06666667  0.        ]
 [ 0.          0.2         0.        ]]

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## Questão 3

In [66]:
def pivot_partial(x):
    return np.argmax(abs(x))

def ref(A, tol=None, pivot=pivot_partial):
    T = np.array(A).astype(float)
    n_linhas, n_colunas = np.shape(T)

    tol = 2**(-52) * np.max(abs(T)) * max(n_linhas, n_colunas)

    n_pivos = 0
    j = 0

    while (j < n_colunas and n_pivos < n_linhas):
        #Encontra o pivô
        p = pivot(T[n_pivos:, j]) + n_pivos

        if abs(T[p, j]) > tol:
            #Encontramos um pivô.

            # Troca a linha p com a n_pivos, caso necessário
            if p != n_pivos:
                for k in range(j, n_colunas):
                    temp = T[p, k]
                    T[p, k] = T[n_pivos, k]
                    T[n_pivos, k] = temp

                p = n_pivos

            for k in range(p + 1, n_linhas):
                multiplicador = T[k, j] / T[p, j]
                for s in range(j+1,n_colunas):
                    T[k, s] -= multiplicador * T[p, s]
                T[k, j] = 0

            n_pivos += 1
        j += 1

    return T

In [67]:
def determinante_matriz (A):

    A_esc = ref(A)

    det = 1

    for i in range(len(A)):
        det *= A_esc[i,i]

    return det

In [68]:
determinante_matriz(M)

2580.0

## Questão 3.5

In [69]:
def DETERMINANTE_LAPLACE (A):
    n = len(A)

    det = 0

    if n == 1:
        return A[0][0]
    
    for j in range(n):
        A_j = []
        for a in range (1, n):
            lista = []
            for b in range(n):
                if b != j:
                    lista.append(A[a][b])
            A_j.append(lista)
        det += (-1)**j * A[0][j] * DETERMINANTE_LAPLACE(A_j)
    
    return det

In [70]:
DETERMINANTE_LAPLACE(M)

2580

## Questão 4

In [71]:
def vetor_ker (A):
    # Define o tamanho do array A
    m, n = np.shape(A)

    # Escalona a matriz A
    A_esc = np.array(escalona_inteira(A))

    # Cria a matriz x do tamanho n
    x = np.ones(n)

    # Vai calcular os valores de x para que Ax = 0
    for i in range(m-1, -1, -1):
        x[i] = (-sum([A_esc[i][k]*x[k] for k in range(i+1, n)]))/A_esc[i][i]
    
    # Calcula a norma do vetor x
    norma_x = np.sqrt(sum([c*c for c in x]))

    # Reescreve o array x normalizado
    x_normalizado = np.array([x[i]/norma_x for i in range(n)])

    return x_normalizado

In [72]:
N = [[4, 5, 9, 7, 9],
     [-2, 6, 7, 0, -3],
     [-9, 7, 0, 5, 2]]

x = np.array(vetor_ker(N))

print(x)

print(N@x)

[-0.49728205 -0.70725558  0.49323545  0.06789294  0.06789294]
[1.55431223e-15 1.77635684e-15 0.00000000e+00]
